In [1]:
import uproot
import awkward as ak
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import hist
import vector
import os
import subprocess
import gc
print("uproot version", uproot.__version__)
print("awkward version", ak.__version__)
print("numpy version", np.__version__)
print("matplotlib version", matplotlib.__version__)
print("hist version", hist.__version__)
print("vector version", vector.__version__)
#print("os version", os.__version__)
#print("subprocess version", subprocess.__version__)
#print("gc version", gc.__version__)

uproot version 5.0.12
awkward version 2.4.3
numpy version 1.25.2
matplotlib version 3.8.0
hist version 2.7.2
vector version 1.1.1


In [2]:
vector.register_awkward() 

In [3]:
DATATYPE = "mc"
assert((DATATYPE == "mc") or (DATATYPE == "data"))
BASEDIR = "/pbs/throng/training/nantes-m2-rps-exp/data" # basedir where to look for runXXX.DATATYPE.root files
IS_MC = True if DATATYPE == "mc" else False

In [4]:
def data_file_path(run,is_mc=IS_MC,dest=BASEDIR):
    datatype="mc" if is_mc else "data"
    print({dest},"/run",{run},".",{datatype},".root")
    return f"{dest}/run{run}.{datatype}.root"

In [5]:
RUNS = [290323, 290327, 290848, 291361, 291360, 291362, 290853, 290860, 291373, 290374, 290375, 291399,
               291400, 290894, 290895, 290404, 291943, 291944, 291948, 291953, 290932, 290423, 291447, 290935, 
               290425, 290427, 291451, 291453, 291976, 291982, 290456, 290458, 290459, 291482, 291485, 290975, 
               290980, 290469, 292012, 291002, 291003, 291004, 291005, 290501, 292040, 292060, 292061, 292062, 
               291041, 290539, 290540, 292075, 292077, 292080, 290549, 290553, 291590, 292106, 292108, 292109, 
               292115, 290590, 291618, 291622, 291624, 292140, 290612, 292160, 292162, 292163, 292164, 292166, 
               290632, 291657, 292168, 292192, 290658, 290660, 291690, 291692, 291694, 291698, 291706, 290687, 
               290692, 290696, 290699, 292242, 292265, 291755, 292269, 292270, 291760, 292273, 292274, 290742, 
               291769, 291263, 290764, 290766, 291283, 291284, 291285, 291795, 291796, 290776, 291803, 290787]
RUNS1 = [290323, 290327, 290848]
SAMPLE_RUNS = sorted(RUNS1)
print(len(SAMPLE_RUNS))

3


In [6]:
file = uproot.open(data_file_path(SAMPLE_RUNS[0],IS_MC))
events = file["eventsTree"]
gen = file["genTree"]
events.show()
events.num_entries
gen.show()
gen.num_entries

{'/pbs/throng/training/nantes-m2-rps-exp/data'} /run {290323} . {'mc'} .root
name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
runNumber            | int32_t                  | AsDtype('>i4')
xVtx                 | double                   | AsDtype('>f8')
yVtx                 | double                   | AsDtype('>f8')
zVtx                 | double                   | AsDtype('>f8')
isCINT               | bool                     | AsDtype('bool')
isCMSL               | bool                     | AsDtype('bool')
isCMSH               | bool                     | AsDtype('bool')
isCMLL               | bool                     | AsDtype('bool')
isCMUL               | bool                     | AsDtype('bool')
nMuons               | int32_t                  | AsDtype('>i4')
Muon_E               | std::vector<float>       | AsJagged(AsDtype('>f4'), he...
Muon_Px              | st

70000

In [17]:
def mag(Px, Py, Pz):
    return np.sqrt(Px*Px + Py*Py + Pz*Pz)

def rest_mass(E, Px, Py, Pz):
    P = mag(Px, Py, Pz)
    return np.sqrt(np.abs(E*E - P*P))


def getTracks(events):
    return ak.zip({"px":events["Muon_Px"],
                       "py":events["Muon_Py"],
                       "pz":events["Muon_Pz"],
                       "E":events["Muon_E"],
                       "MC_label":events["Muon_MCLabel"],
                       "charge":events["Muon_Charge"],
                       "thetaAbs":events["Muon_thetaAbs"],
                       "matched":events["Muon_matchedTrgThreshold"]},
                    with_name='Momentum4D')
"""
def getTracks_gen(gen):
    return ak.zip({"pxGen":gen["Muon_GenPx"],
                       "pyGen":gen["Muon_GenPy"],
                       "pzGen":gen["Muon_GenPz"],
                       "EGen":gen["Muon_GenE"],
                       "MC_Genlabel":gen["Muon_GenLabel"],
                       "MC_GenMotherPDG":gen["Muon_GenMotherPDG"]},
                    with_name='Momentum4D')"""


def scan(dataDescription, 
              hMag:hist.Hist, hPhi:hist.Hist, hEta:hist.Hist, hPt:hist.Hist, hMass:hist.Hist, hMass_OS:hist.Hist, hMass_LS:hist.Hist,
              eventSelector = lambda x:[True]*len(x),
              trackSelector = lambda x:[True]*len(x), 
              #genSelector = lambda x:[True]*len(x), 
              verbose:bool = False):
    Entries = goodEntries = 0
    for batch in uproot.iterate(dataDescription,
                                ["isCINT", "Muon_MCLabel","isCMUL", "isCMSL", "nMuons", "Muon_Px", "Muon_Py", "Muon_Pz", "Muon_E", "Muon_Charge", "Muon_thetaAbs", "Muon_matchedTrgThreshold",],                                
                                report = True):
        events = batch[0] # batch[1] is the report info
        if len(events) < 1000:
            print("something is wrong",batch[1]) # this is a protection for some corrupted input data files 
            break
            
        goodEvents = events[eventSelector(events)] 
        tracks = getTracks(events)
        goodTracks = tracks[trackSelector(tracks)]
        #generation = getTracks_gen(events)
        #goodGen = generation[genSelector(generation)]
        
        hMag.fill(ak.flatten(goodTracks.p))
        hPhi.fill(ak.flatten(goodTracks.phi))
        hEta.fill(ak.flatten(goodTracks.eta))
        hPt.fill(ak.flatten(goodTracks.pt))
        hMass.fill(ak.flatten(goodTracks.mass))
        
        pairs = ak.combinations(goodTracks, 2)
        Px = pairs["0"].px + pairs["1"].px
        Py = pairs["0"].py + pairs["1"].py
        Pz = pairs["0"].pz + pairs["1"].pz
        E = pairs["0"].E + pairs["1"].E
        charge = pairs["0"].charge + pairs["1"].charge
        
        Pair2 = ak.zip({"px":Px,
                       "py":Py,
                       "pz":Pz,
                       "E":E,
                       "charge":charge},
                    with_name='Momentum4D')
        
        hMass_OS.fill(ak.flatten(Pair2[Pair2.charge==0].mass))
        hMass_LS.fill(ak.flatten(Pair2[Pair2.charge!=0].mass))
        
        Entries += len(events)
        goodEntries += len(ak.flatten(goodTracks))/2
        
        #label = [pairs["0"].MC_label, pairs["1"].MC_label]
        
        if verbose:
            #print("-----------------------------------------------------------")
            #print(batch[1], "\nEntries: ", Entries, "\nJ/Psi: ", int(goodEntries), "\nPDG: ", PDG)
            gc.collect()
        return Entries, goodEntries, PDG

In [18]:
%%time

hMag = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 100, name = '$|p|$'))
hPhi = hist.Hist(hist.axis.Regular(bins = 200, start = -22/7, stop = 22/7, name = '$\phi$'))

hEta = hist.Hist(hist.axis.Regular(bins = 100, start = -6, stop = 0, name = '$\eta$'))
hPt = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 100, name = '$p_T$'))
hMass = hist.Hist(hist.axis.Regular(bins = 100, start = 0, stop = 0.2, name = '$m_{\mu}$'))

hMass_OS = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))
hMass_LS = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))

hMass_OS_cut = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))
hMass_LS_cut = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))

hMass_OS_test = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))
hMass_LS_test = hist.Hist(hist.axis.Regular(bins = 100, start = 1.5, stop = 5, name = '$m_{\mu \mu}$'))


CPU times: user 947 µs, sys: 0 ns, total: 947 µs
Wall time: 958 µs


In [19]:
PDG = []
NTotalJPsi =0
NJPsi=[]
Entries = 0
goodEntries = 0
for run in SAMPLE_RUNS:
    a, b, c = scan(dataDescription = f"/pbs/throng/training/nantes-m2-rps-exp/data/run{run}.mc.root:eventsTree",
          hMag = hMag, hPhi = hPhi, hEta = hEta, hPt = hPt, hMass = hMass, hMass_OS = hMass_OS_test, hMass_LS = hMass_LS_test,
          eventSelector = lambda x: x["isCMUL"] == True, 
          trackSelector = lambda x: (((x.thetaAbs < 3) & (x.eta > -4)) & (x.eta < -2.5)), verbose = True)
    Entries += a;
    goodEntries += b;
    NJPsi.append(int(b))
    NTotalJPsi += goodEntries
    
    #print(a,b)
#print("-----------------------------------------------------------", "\nEntries: ", Entries, "\nJ/Psi: ", int(goodEntries), "\nPDG: ", PDG)
print(NTotalJPsi)
print(NJPsi)
print(len(NJPsi))

32819.0
[7349, 637, 9496]
3


In [ ]:
hMag.plot(color = "blue", label = "no cut")
plt.yscale('log')
plt.grid(True)
plt.legend()

In [ ]:
nofEvents = 0
nofTracks=0
m = events.arrays(["nMuons","Muon_Px","Muon_Py","Muon_Pz","Muon_Charge", "Muon_MCPDGCode"],how="zip",entry_stop=150000)

dimuons = ak.flatten([m["nMuons"]==2])
X=np.arange(events.num_entries)[m["nMuons"]==2] #indices dimuons

print(dimuons[3])
print(dimuons)
#for event in m[m.nMuons>0]:
for event in X : 
    nofEvents+=1
    tracks = m["Muon"].to_list()
    for t in tracks:
        nofTracks+=1
        #muons = [abs(t["Muon_MCPDGCode"])==13]
        
print(nofEvents)
print(nofTracks)
print(tracks)
print(m[1:5].to_list())

print(X)
print(len(X))